In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-08-17 11:25:58--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  2.56MB/s    in 0.4s    

2024-08-17 11:25:59 (2.56 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(f"length = {len(text)}")
print(text[:100])

length = 1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [2]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [3]:
# character-level language model
# we tokenize based on characters

# encode chars -> ints
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hi there"))
print(decode(encode("hi there")))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [4]:
# encode entire dataset
import torch
data = torch.tensor(encode(text), dtype=torch.long)
device = torch.device("mps")
data = data.to(device)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [5]:
# tts
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [6]:
# we train on chunks at a time. chunks have a maximum length
block_size = 256
train_data[:block_size + 1]
# append one (get nine characters) because we are making eight predictions, 
# including on the unseen (ninth) char, excluding the first.

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [7]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

In [8]:
torch.manual_seed(1337)
batch_size = 64 # minibatch
block_size = 256 # context
n_embd = 384 # hidden layer dimension

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,), device=device)
    x = torch.stack([data[i : i + block_size] for i in ix]).to(device)
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix]).to(device)
    return x, y

xb, yb = get_batch('train')
print(xb.shape, xb)
print(yb.shape, yb)

torch.Size([64, 256]) tensor([[30, 37,  1,  ..., 52, 42, 43],
        [53, 59, 56,  ..., 45,  1, 53],
        [ 1, 57, 46,  ..., 46, 47, 57],
        ...,
        [39, 58,  1,  ..., 53, 58,  1],
        [47, 52,  1,  ...,  1, 51, 39],
        [ 5, 58, 47,  ..., 32, 46, 43]], device='mps:0')
torch.Size([64, 256]) tensor([[37,  1, 34,  ..., 42, 43, 42],
        [59, 56,  1,  ...,  1, 53, 44],
        [57, 46, 53,  ..., 47, 57,  1],
        ...,
        [58,  1, 51,  ..., 58,  1, 40],
        [52,  1, 58,  ..., 51, 39, 63],
        [58, 47, 57,  ..., 46, 43, 52]], device='mps:0')


In [9]:
@torch.no_grad()
def estimate_loss(model, eval_iters):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
# simplest possible neural network
# bigram!
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class Bigram(nn.Module):
    def __init__(self):
        super().__init__()
        
        # gives us token embeddings
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd, device=device)

        # embedding corresponding to the position of a token
        self.position_embedding_table = nn.Embedding(block_size, n_embd, device=device)
        
        # "language model head"
        self.lm_head = nn.Linear(n_embd, vocab_size, device=device)
        
    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (batchsize, ntoks) tensors
        # for each token, we are predicting the probabilities of the next tokens
        tok_emb = self.token_embedding_table(idx) # (batchsize, ntoks, embd)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (bs, embd)
        
        x = tok_emb + pos_emb 
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # unpack the token predictions
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop to the last #block_size tokens
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            # look at the last time step to predict what comes next
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim=1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = Bigram()
logits, loss = m(xb, yb)
print(xb.shape)
print(yb.shape)
# print(out.shape)
print(loss.item())
print(f"n_params = {sum(p.nelement() for p in m.parameters())}")

torch.Size([64, 256])
torch.Size([64, 256])
4.5627923011779785
n_params = 148289


In [11]:
# expected loss should be random -- prob is uniform
import numpy as np
uniform_prob = 1 / vocab_size
-np.log(uniform_prob)

np.float64(4.174387269895637)

In [12]:
# the batch. holds a zero. kicks off the generation here
idx = torch.zeros((1, 1), dtype = torch.long, device=device)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


U3mpd!Jj-fxcWkPAQbRe.OOOdUfZE&ewKyjk$cH-k3VCOIi'?gozgZeA
pCOrsvMkBcAOyDXOHHN,j&?ofeOAvrYotKyL?DJ&JbL


In [13]:
from time import time_ns

In [158]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) # usually, 3e-4 but for stupid networks you can learn faster
batch_size = 32
DTbar = 0

for steps in range(10_000):
    Ta = time_ns()
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    DT = time_ns() - Ta
    DTbar = (DTbar*steps + DT)/(steps+1)

    if steps % 1_000 == 0:
        print(f"{steps:7d}/{10_000:7d}: loss={loss.item():1.4f}, Tavg={DTbar/1e9:.5f}")

print(loss.item())

      0/  10000: loss=4.5600, Tavg=0.28380


KeyboardInterrupt: 

In [ ]:
idx = torch.zeros((1, 1), dtype = torch.long)
print(decode(m.generate(idx, max_new_tokens=1000)[0].tolist()))

# self attention

In [159]:
# toy ex
torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch=4, time=8, channels=2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [40]:
# information should flow forwards only: e.g. token at position n can receive info from positions [0, n-1]
# simplest way of doing this is to take an average of all the previous tokens
xbow = torch.zeros((B, T, C)) # bag of words, averaged prev tokens
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)

In [41]:
xbow.shape

torch.Size([4, 8, 2])

In [42]:
# this is inefficient. an just matmul
torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print(a)
print(b)
print(c)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [51]:
# we can just build a diagonal matrix
at = torch.tril(torch.ones(3, 3)) 
at @ b

tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])

In [52]:
# tril @ (T, B, ...) -> (T, B, ...)
# now we can get averages
(at / torch.sum(at, 1, keepdim=True)) @ b

tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])

In [59]:
T_ones = torch.ones(T, T)
T_tril = torch.tril(T_ones)
T_avgk = T_tril / torch.sum(T_tril, dim=1, keepdim=True)

In [67]:
xbow2 = T_avgk @ x
print(xbow[xbow != xbow2])
print(xbow2[xbow != xbow2])
print(torch.allclose(xbow, xbow2))

tensor([-0.3199, -0.0396, -0.0682,  0.0084,  0.0020,  0.0712, -0.1128,  0.2527,
        -0.0436, -0.2593, -0.3015,  0.4954,  0.3420,  1.1401, -0.4462,  1.0870,
        -0.4071, -0.1641])
tensor([-0.3199, -0.0396, -0.0682,  0.0084,  0.0020,  0.0712, -0.1128,  0.2527,
        -0.0436, -0.2593, -0.3015,  0.4954,  0.3420,  1.1401, -0.4462,  1.0870,
        -0.4071, -0.1641])
True


In [70]:
# can use softmax
# it basically tells us how significantly tokens interact
# --> "affinities" for certain tokens in the past
tril = torch.tril(torch.ones(T, T)) # lower triangular ones
wei = torch.zeros((T, T)) # all zeros
wei = wei.masked_fill(tril == 0, float('-inf')) # fill the upper triangular portion with -inf
wei = F.softmax(wei, dim=-1) # softmax along every row -> .exp().div(sum(dim=1))
torch.allclose(wei @ x, xbow2)

True

lower triangular multiplication gives us how much affinity a token in the ith row has for the previous rows

In [74]:
# toy ex
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch=4, time=8, channels=2
x = torch.randn(B, T, C)

tril = torch.tril(torch.ones(T, T)) # lower triangular ones
wei = torch.zeros((T, T)) # all zeros
wei = wei.masked_fill(tril == 0, float('-inf')) # fill the upper triangular portion with -inf
wei = F.softmax(wei, dim=-1) # softmax along every row -> .exp().div(sum(dim=1))
out = wei @ x
out.shape

torch.Size([4, 8, 32])

In [75]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [76]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

every token at every position gives two vectors : a query, and a key. the query vector is "what am i looking for", and the key is "what do i contain"

affinities: dot product between keys and query -> dot product gives weights

In [83]:
# toy ex
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch=4, time=8, channels=3 2
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)

# key learns what the token contains
# query is what the token wants

# all queries dot all the keys
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) -> (B, T, T). 
# We now get a T x T matrix containing the weights

tril = torch.tril(torch.ones(T, T)) # lower triangular ones
wei = wei.masked_fill(tril == 0, float('-inf')) # fill the upper triangular portion with -inf
wei = F.softmax(wei, dim=-1) # softmax along every row -> .exp().div(sum(dim=1))

# x is "private info" to the token
# v is the thing that is aggregated between nodes in a given window
v = value(x)

out = wei @ v
#out = wei @ x
out.shape

torch.Size([4, 8, 16])

In [ ]:
# note that wei has no notion of position
# this is why we encode positions also

# we also may not need the tril masking: here we use it for autoregressive text generation
# for other use cases, you could unmask it

# "self-attention" is looking within the context as Q, K, V come from x.
# "cross-attention" is when a separate source of information is used

# "scaled" attention normalizes by sqrt(head size)
# to keep the variance of the weight matrix diffuse and disallow softmax from saturating

In [88]:
(torch.randn(B, T, head_size) @ torch.randn(B, T, head_size).mT * (head_size**-.5)).var()

tensor(0.9957)

In [84]:
v.shape

torch.Size([4, 8, 16])

In [80]:
# wei is now data dependent
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [82]:
wei[0, -1]

tensor([0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391],
       grad_fn=<SelectBackward0>)

# attention

In [21]:
n_layer = 6
n_head = 6
dropout = 0.2

In [22]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embd, head_size, bias=False)        
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x) # (B, T, C)
        q = self.query(x) # (B, T, C)
        # compute self-attention = QK^T / sqrt(head_size)
        # (B, T, C) @ (B, C, T) --> (B, T, T)
        wei = q @ k.mT * C**-0.5
        # drop out the forward connections
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # LT (B, T, T)
        # softmax to get multinomials
        wei = F.softmax(wei, dim=-1)

        wei = self.dropout(wei)

        v = self.value(x)
        out = wei @ v # (B, T, T) @ (B, T, C) --> (B, T, C)
        return out

In [16]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

In [17]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),   
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)

In [18]:
class LayerNorm(nn.Module):
    def __init__(self, dim, eps=1e-5):
        self.eps = eps
        self.gamma = torch.ones(dim, device=device)
        self.beta = torch.zeros(dim, device=device)
    def forward(self, x):
        xmean = x.mean(1, keepdim=True) # "layer" mean: across each context window
        xvar = x.var(1, keepdim=True)
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # zscore
        self.out = self.gamma * xhat + self.beta
        return self.out

In [19]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        assert(head_size * n_head == n_embd)
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self, x):
        # residual connections by adding self 
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [23]:
# simplest possible neural network
# bigram!
torch.manual_seed(1337)
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        # gives us token embeddings
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd, device=device)

        # embedding corresponding to the position of a token
        self.position_embedding_table = nn.Embedding(block_size, n_embd, device=device)
        
        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head=n_head) for _ in range(n_layer)]
        )

        self.ln_f = nn.LayerNorm(n_embd)
        
        # language output
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        
    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (batchsize, ntoks) tensors
        # for each token, we are predicting the probabilities of the next tokens
        tok_emb = self.token_embedding_table(idx) # (batchsize, ntoks, embd)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (bs, embd)
        
        x = tok_emb + pos_emb 

        # invoke self-attention
        x = self.blocks(x)
        # layernorm before logits
        x = self.ln_f(x)
        
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # unpack the token predictions
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop to the last #block_size tokens
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            # look at the last time step to predict what comes next
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim=1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel()
optimizer = torch.optim.AdamW(m.parameters(), lr=3e-4) # usually, 3e-4 but for stupid networks you can learn faster

steps = 0
m = m.to(device)
logits, loss = m(xb, yb)
print(xb.shape)
print(yb.shape)
# print(out.shape)
print(loss.item())
print(f"n_params = {sum(p.nelement() for p in m.parameters())}")

torch.Size([64, 256])
torch.Size([64, 256])
4.389363765716553
n_params = 10788929


In [29]:
#torch.manual_seed(9999)
batch_size = 64
max_s = 15_000
DTbar = 0.0
valloss = float('inf')
print(f"{steps:7d}/{max_s:7d}: loss={loss.item():1.4f}, valloss={estimate_loss(m, 20)["val"]:.4f}, Tavg={DTbar/1e9:.5f}")
while steps < max_s:
    steps += 1
    Ta = time_ns()
    
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    DT = time_ns() - Ta
    DTbar = (DTbar*steps + DT)/(steps+1)

    if steps % 10 == 0:
        print(f"{steps:7d}/{max_s:7d}: loss={loss.item():1.4f}, valloss={estimate_loss(m, 20)["val"]:.4f}, Tavg={DTbar/1e9:.5f}")
    
print(f"{max_s:7d}/{max_s:7d}: loss={loss.item():1.4f}, Tavg={DTbar/1e9:.5f}")

   2080/  15000: loss=1.4147, valloss=1.5672, Tavg=0.00000
   2090/  15000: loss=1.3972, valloss=1.5757, Tavg=0.00358
   2100/  15000: loss=1.3903, valloss=1.5774, Tavg=0.00705
   2110/  15000: loss=1.4202, valloss=1.5646, Tavg=0.01049
   2120/  15000: loss=1.4031, valloss=1.5801, Tavg=0.01390
   2130/  15000: loss=1.3890, valloss=1.5784, Tavg=0.01729
   2140/  15000: loss=1.4034, valloss=1.5662, Tavg=0.02064
   2150/  15000: loss=1.4331, valloss=1.5737, Tavg=0.02395
   2160/  15000: loss=1.4193, valloss=1.5627, Tavg=0.02724
   2170/  15000: loss=1.3971, valloss=1.5730, Tavg=0.03049
   2180/  15000: loss=1.3843, valloss=1.5673, Tavg=0.03372
   2190/  15000: loss=1.3772, valloss=1.5613, Tavg=0.03691
   2200/  15000: loss=1.4180, valloss=1.5633, Tavg=0.04007
   2210/  15000: loss=1.4065, valloss=1.5623, Tavg=0.04323
   2220/  15000: loss=1.4035, valloss=1.5511, Tavg=0.04635
   2230/  15000: loss=1.3814, valloss=1.5567, Tavg=0.04943
   2240/  15000: loss=1.4013, valloss=1.5498, Tavg=0.052

In [30]:
estimate_loss(m, 20)

{'train': tensor(0.4973), 'val': tensor(1.9065)}

In [31]:
steps

15000

In [37]:
prompt = torch.zeros((1, 1), dtype = torch.long, device=device)

In [38]:
from IPython.display import display, Code, clear_output

while True:
    prompt = m.generate(prompt, max_new_tokens=1)
    clear_output(wait=True)
    display(Code(language='text', data=decode(prompt[0].tolist())))

Tell my most greeting arm our request
Is passing correction than the day of alliants,
Brought the Englishmanen Hump of Henrage to
open the Lordshire of Rome, whose wrongs purpose,
And gave comfort! Whom w then? I have could I had on
Shall on me wrong, too rouse to the Tower.
And she must live to Angelo, as I have,
And leave unsafely to thine, I beseech you,
Honesty nature for your beings to yourself;
For now I am you with, but that dispatch:
Mend Yirkshire came my very stumper'd:
And now 'twould say 'I that give us heart
And long be castor'd Henry's goodly queen,
For that she stirs hath been an achieve's eyes,
Along heavy burree in jest.

KING RICHARD II:
All purchased, that still is she laid,
His golty mind the Phoetra conspirates
Was never to command, or on armorant foe,
One kingdon'd up by chanliness, not took damn'd
Deperseth end.

BRUTUS:
Be thus death'd i' the matter:
Life, away!

Constable, nurse,nes, coward!

CAMILLO:

MENENIUS:
Hence, a brothelm!
Why, you are comfortable. What is this?

VOLUMNIA:
Not for my best good fast, not yourself, I crept,
That in the yur consuls wonder of Calais.

CLUDIO:
The best knows you to cunning t

KeyboardInterrupt: 